# EDA de correlación de datos y modelo de regresión lineál

In [21]:
# Manejo de archivos
import os
from pathlib import Path

# Manejo de datos
import pandas as pd
import numpy as np

# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

#datetime
from datetime import datetime, timedelta

## Cargar datos

In [4]:
print(f"Current path: {os.getcwd()}")

Current path: /home/misael/repos/MLOps-grinding/notebooks


In [5]:
# Definir la ruta de los datos
data_path: Path = Path(r"../data/raw")
if not data_path.exists():
    print(f"Path {data_path} does not exits.")

# Imprimir archivos en la carpeta de datos
print(list(data_path.glob("*")))

[PosixPath('../data/raw/datos_molienda.parquet')]


In [8]:
df = pd.read_parquet(data_path / "datos_molienda.parquet")

In [9]:
df.head()

,velocidad,potencia,celda_carga,pad_1,pad_2,pad_3,pad_4,pad_5,pad_6,pad_7,...,in1_5,in1,f80,dureza,per_solidos,wi,spi,imp_criticos,imp_estandares,jb
date,,,,,,,,,,,,,,,,,,,,,
2022-11-15 00:00:00,0.000000,-0.482877,-0.074890,0.816327,-0.794444,-0.545455,0.424731,0.175926,0.515464,0.688525,...,0.465352,0.724886,-0.114582,1.107937,-0.153193,0.246579,0.518816,0.5,0.259414,-1.544362
2022-11-15 00:00:10,0.000000,-0.498288,-0.050831,0.816327,-0.538889,0.831169,0.586022,0.175926,-0.072165,-0.131148,...,0.587188,0.784246,-0.114582,1.107937,-0.227922,0.246579,0.518816,0.5,0.259414,-1.544638
2022-11-15 00:00:20,0.000000,-0.839041,-0.027548,0.843537,-0.538889,0.831169,0.586022,0.888889,0.814433,-0.131148,...,0.587188,0.784246,-0.114582,1.107937,-0.297216,0.246579,0.518816,0.5,0.259414,-1.544928
2022-11-15 00:00:30,0.000000,-0.386986,-0.027548,0.455782,-0.255556,0.077922,0.091398,0.500000,0.814433,-0.172131,...,0.587188,0.784246,-0.303652,1.107937,-0.299254,0.246579,0.518816,0.5,0.259414,-1.545215
2022-11-15 00:00:40,-0.166669,-0.535959,-0.008146,0.455782,0.133333,0.103896,0.000000,0.500000,0.257732,0.008197,...,0.374744,0.478899,-0.313486,1.107937,-0.299254,0.246579,0.518816,0.5,0.259414,-1.545534


In [10]:
df.shape

(39586, 35)

Tenemos 35 variables, la variable objetivo será la `presion`.

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 39586 entries, 2022-11-15 00:00:00 to 2022-11-20 00:00:00
Data columns (total 35 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   velocidad       39586 non-null  float64
 1   potencia        39586 non-null  float64
 2   celda_carga     39586 non-null  float64
 3   pad_1           39586 non-null  float64
 4   pad_2           39586 non-null  float64
 5   pad_3           39586 non-null  float64
 6   pad_4           39586 non-null  float64
 7   pad_5           39586 non-null  float64
 8   pad_6           39586 non-null  float64
 9   pad_7           39586 non-null  float64
 10  pad_8           39586 non-null  float64
 11  rendimiento     39586 non-null  float64
 12  flujo_agua      39586 non-null  float64
 13  ruido           39586 non-null  float64
 14  giro            39586 non-null  float64
 15  p80             39586 non-null  float64
 16  in14            39586 non-null  float64
 

Todos los campos son de tipo **float** y **no** tenemos datos faltantes.

In [15]:
y_hat = df.iloc[-1].values

In [23]:
df.loc[datetime(2022,11,20,10), :] = y_hat

In [24]:
df

,velocidad,potencia,celda_carga,pad_1,pad_2,pad_3,pad_4,pad_5,pad_6,pad_7,...,in1_5,in1,f80,dureza,per_solidos,wi,spi,imp_criticos,imp_estandares,jb
date,,,,,,,,,,,,,,,,,,,,,
2022-11-15 00:00:00,0.000000,-0.482877,-0.074890,0.816327,-0.794444,-0.545455,0.424731,0.175926,0.515464,0.688525,...,0.465352,0.724886,-0.114582,1.107937,-0.153193,0.246579,0.518816,0.500000,0.259414,-1.544362
2022-11-15 00:00:10,0.000000,-0.498288,-0.050831,0.816327,-0.538889,0.831169,0.586022,0.175926,-0.072165,-0.131148,...,0.587188,0.784246,-0.114582,1.107937,-0.227922,0.246579,0.518816,0.500000,0.259414,-1.544638
2022-11-15 00:00:20,0.000000,-0.839041,-0.027548,0.843537,-0.538889,0.831169,0.586022,0.888889,0.814433,-0.131148,...,0.587188,0.784246,-0.114582,1.107937,-0.297216,0.246579,0.518816,0.500000,0.259414,-1.544928
2022-11-15 00:00:30,0.000000,-0.386986,-0.027548,0.455782,-0.255556,0.077922,0.091398,0.500000,0.814433,-0.172131,...,0.587188,0.784246,-0.303652,1.107937,-0.299254,0.246579,0.518816,0.500000,0.259414,-1.545215
2022-11-15 00:00:40,-0.166669,-0.535959,-0.008146,0.455782,0.133333,0.103896,0.000000,0.500000,0.257732,0.008197,...,0.374744,0.478899,-0.313486,1.107937,-0.299254,0.246579,0.518816,0.500000,0.259414,-1.545534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-19 23:59:10,-0.166669,0.130137,-0.343421,-0.639456,-0.266667,0.415584,-0.360215,0.000000,-0.659794,-0.237705,...,-0.081994,-0.014469,0.178132,-0.375919,0.617527,1.216499,0.156551,0.673077,0.769874,1.428620
2022-11-19 23:59:20,-0.166669,0.534247,-0.343421,-0.639456,-1.050000,-0.564935,0.193548,-0.351852,-0.546392,-0.532787,...,0.116896,0.302638,0.166967,-0.375919,0.633151,1.216499,0.156551,0.673077,0.769874,1.428344
2022-11-19 23:59:40,0.000000,0.400685,-0.343421,0.693878,-0.655556,-0.409091,-0.822581,-0.500000,-0.206186,0.008197,...,0.116896,0.302638,0.145740,-0.375919,0.640963,1.216499,0.156551,0.673077,0.769874,1.427784


In [26]:
pwd

'/home/misael/repos/MLOps-grinding/notebooks'

In [29]:
main_folder = Path.cwd().parent / 'data'
main_folder

PosixPath('/home/misael/repos/MLOps-grinding/data')

In [30]:
df.to_parquet(main_folder / 'processed' /'fome1.parquet')